In [ ]:
import os
import pandas as pd
import zipfile as z
import requests as r
import json
import shutil
import collections

In [ ]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 300

In [ ]:
files = os.listdir("./devicezip")

In [ ]:
messages = pd.read_csv('messages.csv', names=['survey_instance', 'upload_date', 'status', 'messages'])
messages = messages.dropna(subset=['messages'])
messages = messages[messages['messages'].str.contains('\[\]')]

In [ ]:
def createFileType(x):
    val = x.split('.')[1]
    return val.split(' ')[0].lower()

def createFileName(x):
    return x.split(' ')[0]

In [ ]:
messages['file_type'] = messages['messages'].apply(createFileType)
messages['file_name'] = messages['messages'].apply(createFileName)

In [ ]:
messages

In [ ]:
filenames = messages.groupby(['file_name']).agg('count').reset_index()

In [ ]:
filenames

In [ ]:
sources = []

if os.path.isdir('./sources'):
    shutil.rmtree('./sources')
    os.mkdir('./sources')
    
for file in files:
    location = "./sources/" + file.split('.')[0]
    os.mkdir(location)
    with z.ZipFile('./devicezip/'+file, 'r') as zip_ref:
        zip_ref.extractall(location)
    
    with open("{}/data.json".format(location) , 'r') as data:
        data = json.load(data)
        sources.append(data)
    
    shutil.rmtree(location)

In [ ]:
datapoints = pd.DataFrame(sources)
datapoints['responses_string'] = datapoints['responses'].astype('str')

In [ ]:
datapoints['responses_string'][0]

In [ ]:
def getFiles(x, filenames):
    listFile = []
    for file in filenames:
        if file in x:
            listFile.append(file)
    return listFile

In [ ]:
datapoints['files'] = datapoints['responses_string'].apply(lambda x: getFiles(x, filenames['file_name'].to_list()))
datapoints['keep'] = datapoints['files'].apply(lambda x: False if len(x) == 0 else True)
datapoints = datapoints[datapoints['keep']]

In [ ]:
datapoints_file = []
collection = datapoints['files'].to_list()
[[datapoints_file.append(x) for x in y] for y in collection]
print("datapoint with missing files  " + str(len(datapoints_file)))
print("the missing files " + str(len(filenames)))

### duplicates
##### duplicates file in datapoint

In [ ]:
duplicates = [item for item, count in collections.Counter(datapoints_file).items() if count > 1]
datapoints['duplicate_file'] = datapoints['responses_string'].apply(lambda x: getFiles(x, duplicates))

In [ ]:
datapoints['keep'] = datapoints['duplicate_file'].apply(lambda x: False if len(x) == 0 else True)
datapoints = datapoints[datapoints['keep']]

In [ ]:
print("the datapoint has same file " + str(len(datapoints)))
print("the duplicate file " + str(len(duplicates)))

In [ ]:
datapoints = datapoints.sort_values('duplicate_file').reset_index()

In [ ]:
[item for item, count in collections.Counter(datapoints['dataPointId'].to_list()).items() if count > 1]

In [ ]:
datapoints['responses'][0]

In [ ]:
from datetime import datetime
date = datetime.fromtimestamp(datapoints['submissionDate'][0]/1000)
print(date)